# SB1 Big Data Downloads
An analysis into the corridor analysis data downloads from Streetlight and Replica

In [1]:
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

import altair as alt
from calitp_data_analysis import calitp_color_palette as cp

from IPython.display import Markdown, HTML, display_html, display
from IPython.core.display import display

import numpy as np

/tmp/ipykernel_305/4150955979.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [119]:
# ! pip install nltk

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import re

In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)

In [5]:
GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/corridor_study_data/"

### Read in Replica Data Spring 2023

In [6]:
replica_trips = "replica-bay_bridge-03_06_24-trips_dataset.csv"

In [7]:
replica_ppl = "replica-bay_bridge-03_06_24-people_dataset.csv"

In [8]:
r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))

/tmp/ipykernel_305/2643890524.py:1: DtypeWarning: Columns (2,4,5,9,10,11,18,25,26,29,30,35,36,40,43,48,50) have mixed types. Specify dtype option on import or set low_memory=False.
  r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))


In [9]:
display(HTML("<h4><strong> Sample of data</strong></h4>"))


In [10]:
r_trips.sample(5)

,origin_building_use,trip_duration_minutes,trip_taker_language,origin_trct_2020,trip_taker_household_id,trip_taker_sex,trip_taker_home_bgrp_2020,origin_st_2020,trip_taker_home_st_2020,trip_taker_race_ethnicity,transit_agency,transit_route,trip_taker_resident_type,vehicle_type,trip_taker_age,activity_id,destination_bgrp_2020,destination_building_use,trip_taker_wfh,destination_st_2020,destination_cty_2020,trip_taker_work_bgrp_2020,destination_trct_2020,trip_taker_household_income,origin_bgrp_2020,trip_taker_commute_mode,trip_taker_available_vehicles,primary_mode,previous_trip_purpose,trip_taker_building_type,transit_submode,trip_taker_work_st_2020,trip_taker_household_size,origin_cty_2020,destination_land_use,trip_taker_industry,trip_taker_tenure,vehicle_fuel_type,trip_taker_home_trct_2020,trip_taker_work_trct_2020,trip_taker_education,trip_start_time,trip_taker_individual_income,trip_taker_employment_status,trip_purpose,origin_land_use,trip_taker_work_cty_2020,trip_distance_miles,trip_taker_school_grade_attending,trip_taker_home_cty_2020,trip_taker_person_id,trip_end_time
41062,multi_family,39,english,"4026 (Alameda, CA)",9865764514976087001,female,"2 (Tract 4008, Alameda, CA)",California,California,white_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,32.0,13572652705366288897,"2 (Tract 231.03, San Francisco, CA)",civic_institutional,in_person,California,San Francisco,"2 (Tract 231.03, San Francisco, CA)","231.03 (San Francisco, CA)",106510.0,"1 (Tract 4026, Alameda, CA)",private_auto,two,auto_passenger,social,several_units,NaN,California,2.0,Alameda,civic_institutional,naics62,renter,unknown_fuel_type,"4008 (Alameda, CA)","231.03 (San Francisco, CA)",advanced_degree,05:44:00,39117.0,employed,work,multi_family,San Francisco,14.7,not_attending_school,Alameda,12411714210989382867,06:23:51
521961,multi_family,30,english,"4036 (Alameda, CA)",9002566160742779172,male,"3 (Tract 4036, Alameda, CA)",California,California,black_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,47.0,2272639320515876334,"2 (Tract 607.01, San Francisco, CA)",healthcare,in_person,California,San Francisco,"2 (Tract 607.01, San Francisco, CA)","607.01 (San Francisco, CA)",54654.0,"3 (Tract 4036, Alameda, CA)",public_transit,one,auto_passenger,other_activity_type,multiple_units,NaN,California,1.0,Alameda,healthcare,naics621498,renter,unknown_fuel_type,"4036 (Alameda, CA)","607.01 (San Francisco, CA)",some_college,07:08:00,54654.0,employed,work,multi_family,San Francisco,11.7,not_attending_school,Alameda,5728984309377250927,07:38:19
58551,retail,33,asian_pacific,"178.04 (San Francisco, CA)",6611448684125492661,female,"4 (Tract 4330, Alameda, CA)",California,California,asian_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,53.0,3527261137915334837,"4 (Tract 4330, Alameda, CA)",single_family,in_person,California,Alameda,"1 (Tract 117, San Francisco, CA)","4330 (Alameda, CA)",78147.0,"2 (Tract 178.04, San Francisco, CA)",private_auto,three_plus,auto_passenger,shop,single_family,NaN,California,3.0,San Francisco,single_family,naics81,owner,unknown_fuel_type,"4330 (Alameda, CA)","117 (San Francisco, CA)",bachelors_degree,17:54:19,26762.0,employed,home,mixed_use,San Francisco,21.6,not_attending_school,Alameda,17433978243373401972,18:27:57
441579,education,43,asian_pacific,"4287 (Alameda, CA)",14675650994361629494,male,"2 (Tract 610, San Francisco, CA)",California,California,asian_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,26.0,182082892017972511,"2 (Tract 610, San Francisco, CA)",single_family,in_person,California,San Francisco,"1 (Tract 9843, San Mateo, CA)","610 (San Francisco, CA)",135543.0,"1 (Tract 4287, Alameda, CA)",private_auto,two,private_auto,work,single_family,NaN,California,3.0,Alameda,single_family,naics453920,owner,other_non_bev,"610 (San Francisco, CA)","9843 (San Mateo, CA)",some_college,19:41:25,54654.0,employed,home,education,San Mateo,18.1,not_attending_school,San Francisco,4521760078323707356,20:24:40
454281,e

In [11]:
display(HTML("<h4><strong> Columns in Replica Trips Data</strong></h4>"))


In [12]:
r_trips.columns

Index(['origin_building_use', 'trip_duration_minutes', 'trip_taker_language',
       'origin_trct_2020', 'trip_taker_household_id', 'trip_taker_sex',
       'trip_taker_home_bgrp_2020', 'origin_st_2020',
       'trip_taker_home_st_2020', 'trip_taker_race_ethnicity',
       'transit_agency', 'transit_route', 'trip_taker_resident_type',
       'vehicle_type', 'trip_taker_age', 'activity_id',
       'destination_bgrp_2020', 'destination_building_use', 'trip_taker_wfh',
       'destination_st_2020', 'destination_cty_2020',
       'trip_taker_work_bgrp_2020', 'destination_trct_2020',
       'trip_taker_household_income', 'origin_bgrp_2020',
       'trip_taker_commute_mode', 'trip_taker_available_vehicles',
       'primary_mode', 'previous_trip_purpose', 'trip_taker_building_type',
       'transit_submode', 'trip_taker_work_st_2020',
       'trip_taker_household_size', 'origin_cty_2020', 'destination_land_use',
       'trip_taker_industry', 'trip_taker_tenure', 'vehicle_fuel_type',
       't

#### Data Exploration: Replica Trips
* Summarizing the data that we exported from Replica
* Existing visualizations in replica: 
   * Primary Mode
   * Trip Purpose
   * Starting hour 
   * Trip Duration/Distance
   * Origin Destination
   * Vehicle Fuel type
   * Transit Routes/Stops/Sub mode/Agency
   * Household Income
   * Race and Ethnicity
   * Private Auto Availability
   * Age
   * Employment/School Status
   
   

In [13]:
trips = r_trips>>group_by(_.primary_mode)>>summarize(avg_trip_time = _.trip_duration_minutes.mean(),
                                                     number_trips = _.activity_id.nunique())

In [123]:
chart = (
        alt.Chart(trips)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("avg_trip_time"),
            color=alt.Color("avg_trip_time", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)
            ), tooltip=trips.columns.tolist())
      .properties(title = "Average Trip Time",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [124]:
chart = (
        alt.Chart(trips)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("number_trips"),
            color=alt.Color("number_trips", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,)
            ), tooltip=trips.columns.tolist())
      .properties(title="Number of Trips",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [18]:
display(HTML("<h4><strong>Length of Trips dataframe</strong></h4>"))

In [19]:
len(r_trips)

674864

In [20]:
display(HTML("<h4><strong>Number of unique activity ids in data</strong></h4>"))

In [21]:
r_trips>>summarize(number_trips = _.activity_id.nunique())

,number_trips
0,674864


In [22]:
display(HTML("<h4><strong>Number of unique trip taker ids</strong></h4>")) 

In [23]:
r_trips>>summarize(number_trips = _.trip_taker_person_id.nunique())

,number_trips
0,309596


In [24]:
race_ethnicity_by_mode = r_trips>>group_by(_.primary_mode)>>count(_.trip_taker_race_ethnicity)

In [25]:
display(HTML("<h4><strong>Trip Taker Race and Ethnicity by Mode</strong></h4>")) 

In [26]:
chart = (
        alt.Chart(race_ethnicity_by_mode)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("n"),
            color=alt.Color("trip_taker_race_ethnicity", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
                                                                         domain=race_ethnicity_by_mode["trip_taker_race_ethnicity"].unique().tolist())
            ),
            tooltip=race_ethnicity_by_mode.columns.tolist())
            
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [27]:
display(HTML("<H4><strong>Transit Mode Splits</strong></H4>")) 


In [28]:
display(HTML("<strong>Looking at transit mode, need to alter the transit_agency col and transit_submode to get a better count.</strong>")) 


In [155]:
ptt_agency_count = (r_trips
 >>filter(_.primary_mode=="public_transit")
 >>group_by(_.primary_mode, _.transit_submode, _.transit_agency)
 >>summarize(n =_.activity_id.nunique())
 >>arrange(-_.n))

In [156]:
ptt_agency_count

,primary_mode,transit_submode,transit_agency,n
0,public_transit,bus,AC TRANSIT,2788
12,public_transit,"bus, bus","San Francisco Municipal Transportation Agency, AC TRANSIT",632
8,public_transit,"bus, bus","AC TRANSIT, San Francisco Municipal Transportation Agency",516
6,public_transit,"bus, bus","AC TRANSIT, AC TRANSIT",263
75,public_transit,"bus, light_rail","AC TRANSIT, San Francisco Municipal Transportation Agency",253
...,...,...,...,...
128,public_transit,"rail, bus","Caltrain, SamTrans",1
130,public_transit,"rail, bus, bus","Amtrak, AC TRANSIT, AC TRANSIT",1
133,public_transit,"rail, light_rail, bus","Caltrain, San Francisco Municipal Transportation Agency, AC TRANSIT",1
136,public_transit,"rail, subway, bus, bus","Caltrain, Bay Area Rapid Transit, San Francisco Municipal Transportation Agency, AC TRANSIT",1


In [157]:
display(HTML("<h4><strong>Most common transit mode combinations</strong></h4>")) 


In [158]:
ptt_modes = (r_trips
             >>filter(_.primary_mode =="public_transit")
             >>count(_.transit_submode)>>arrange(-_.n))

In [159]:
ptt_modes.head(10)

,transit_submode,n
0,bus,2999
1,"bus, bus",1706
21,"bus, light_rail",316
2,"bus, bus, bus",302
41,"light_rail, bus, bus",127
40,"light_rail, bus",76
55,"subway, bus",66
32,"bus, subway",63
14,"bus, bus, light_rail",49
33,"bus, subway, bus",46


In [160]:
##### unnesting the transit submode and agencies to get counts. 

In [161]:
ptt_agency_count['transit_submode'] = ptt_agency_count['transit_submode'].astype(str)
ptt_agency_count['transit_agency'] = ptt_agency_count['transit_agency'].astype(str)

In [162]:
ptt_agency_count['n_modes_taken'] = ptt_agency_count.transit_submode.apply(lambda x: len(x.split()))

In [163]:
# ptt_agency_count>>arrange(-_.n_modes_taken)

In [164]:
## This line of code gets all agencies listed, even if agency is named twice. 
## Need code that counts the unique agencies
#ptt_agency_count['agency_count'] = ptt_agency_count.transit_agency.apply(lambda x: len(x.split(", ")))

In [165]:
ptt_agency_count['agency_count'] = [len(set(x.split(", "))) for x in
                        ptt_agency_count['transit_agency'].str.lower()]

,primary_mode,transit_submode,transit_agency,n,n_modes_taken,agency_count
0,public_transit,bus,AC TRANSIT,2788,1,1
12,public_transit,"bus, bus","San Francisco Municipal Transportation Agency, AC TRANSIT",632,2,2
8,public_transit,"bus, bus","AC TRANSIT, San Francisco Municipal Transportation Agency",516,2,2
6,public_transit,"bus, bus","AC TRANSIT, AC TRANSIT",263,2,1
75,public_transit,"bus, light_rail","AC TRANSIT, San Francisco Municipal Transportation Agency",253,2,2
...,...,...,...,...,...,...
128,public_transit,"rail, bus","Caltrain, SamTrans",1,2,2
130,public_transit,"rail, bus, bus","Amtrak, AC TRANSIT, AC TRANSIT",1,3,2
133,public_transit,"rail, light_rail, bus","Caltrain, San Francisco Municipal Transportation Agency, AC TRANSIT",1,3,3
136,public_transit,"rail, subway, bus, bus","Caltrain, Bay Area Rapid Transit, San Francisco Municipal Transportation Agency, AC TRANSIT",1,4,4


In [184]:
modes_count = ptt_agency_count>>group_by(_.n_modes_taken)>>summarize(n_trips = _.n.sum())

In [185]:
modes_count

,n_modes_taken,n_trips
0,1,3063
1,2,2246
2,3,644
3,4,70
4,5,9
5,6,1


In [187]:
chart = (
        alt.Chart((modes_count))
        .mark_bar(size=60)
        .encode(
            x=alt.X("n_modes_taken", title ="Number of Modes Taken per Trip"),
            y=alt.Y("n_trips", title = "Number of Trips"),
            color=alt.Color("n_trips", title = "Number of Trips",
                            scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)),
            tooltip=modes_count.columns.tolist())
            
      .properties(title = "How Many Modes are Taken Per Trip",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [193]:
# ptt_agency_count>>group_by(_.agency_count)>>count(_.n_modes_taken)>>arrange(-_.n)
agency_mode_trips = ptt_agency_count>>group_by(_.agency_count, _.n_modes_taken)>>summarize(ntrips=_.n.sum())>>arrange(-_.ntrips)

In [216]:
chart = (
        alt.Chart((agency_mode_trips))
        .mark_circle(size=100)
        .encode(
            x=alt.X("agency_count", title ="Number of Agencies"),
            y=alt.Y("n_modes_taken", title = "Number of Modes Taken"),
            color=alt.Color("ntrips", title = "Number of Trips",
                            scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)),
            tooltip=agency_mode_trips.columns.tolist())
            
      .properties(title = "How Many Modes are Taken Per Trip",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [45]:
##### Getting columns for each agency and counts 

In [127]:
# ptt_agency_count['transit_agency'].unique().tolist()

In [198]:
## Get just one columns
column = ptt_agency_count[["transit_agency"]]
#remove single-dimensional entries from the shape of an array
col_text = column.squeeze()
# get list of words
text_list = col_text.tolist()
# #join list of words 
text_list = ', '.join(text_list).title()

In [199]:
text_list = text_list.replace(", ", "', '")

In [200]:
text_list = "['" + text_list + "']"

In [201]:
# text_list

In [202]:
import ast


In [203]:
agency_list = ast.literal_eval(text_list)

In [204]:
agency_list = set(agency_list)

In [205]:
# len(agency_list)

In [206]:
display(HTML("<h4><strong>Agencies Identified in Trips Data</strong></h4>")) 

(agency_list)

{'Ac Transit',
 'Amtrak',
 'Bay Area Rapid Transit',
 'Caltrain',
 'Fairfield And Suisun Transit',
 'Marin Transit',
 'Nan',
 'Sacramento Regional Transit',
 'Samtrans',
 'San Francisco Bay Ferry',
 'San Francisco Municipal Transportation Agency',
 'San Joaquin Regional Transit District (Rtd)',
 'Santa Cruz Metro',
 'Soltrans',
 'The S',
 'Tri Delta Transit',
 'Vta'}

In [56]:
#### using this stackoverflow code to count the instances of the transit agencies in the data

In [57]:
## https://stackoverflow.com/questions/46250505/count-the-number-of-times-multiple-substrings-appear-in-dataframe-column

In [58]:
# ptt_agency_count['transit_agency'] = ptt_agency_count['transit_agency'].str.strip('()').str.split(',')

In [60]:
# ptt_agency_count.join(ptt_agency_count['transit_agency'].str.join('|').str.get_dummies())

In [61]:
### making a copy of the ptt_agency_count
df = ptt_agency_count.copy()

In [62]:
# df['transit_agency'] = df['transit_agency'].astype(str)

In [64]:
# df2 = df['transit_agency'].str.get_dummies(sep=', ')

In [65]:
# df2

##### Trying ChatGPT approach

In [274]:
agency_indv_count = (r_trips>>filter(_.primary_mode=="public_transit")>>select(_.transit_agency))
agency_indv_count['transit_agency'] = agency_indv_count['transit_agency'].astype(str)

In [278]:
# transit_agencies = set()
# for agencies in df['transit_agency'].str.split(', '):
#     transit_agencies.update(agencies)

In [279]:
# unique_agencies = []

In [280]:
# for agency in transit_agencies:
#     df[agency] = df['transit_agency'].str.count(agency)
#     unique_agencies.append(agency)

In [281]:
# ### adding column for unique agencies list
# def get_unique_agencies(agency_list):
#     unique_agencies = set()
#     for agencies in agency_list:
#         unique_agencies.update(agencies.split(', '))
#     return ', '.join(sorted(list(unique_agencies)))

# # Applying the function to each row of the dataframe to get unique agencies
# df['unique_agencies'] = df['transit_agency'].str.split(', ').apply(lambda x: get_unique_agencies(x))

In [286]:
def get_dummies_by_agency(df, col):
    transit_agencies = set()
    for agencies in df[col].str.split(', '):
        transit_agencies.update(agencies)
    unique_agencies = []
    
    for agency in transit_agencies:
        df[agency] = df[col].str.count(agency)
        unique_agencies.append(agency)

    ### adding column for unique agencies list
    def get_unique_agencies(agency_list):
        unique_agencies = set()
        for agencies in agency_list:
            unique_agencies.update(agencies.split(', '))
        return ', '.join(sorted(list(unique_agencies)))

    # Applying the function to each row of the dataframe to get unique agencies
    df['unique_agencies'] = df[col].str.split(', ').apply(lambda x: get_unique_agencies(x))
    
    return df

In [283]:
df = get_dummies_by_agency(df, "transit_agency")

In [284]:
agency_indv_count = get_dummies_by_agency(agency_indv_count, "transit_agency")

In [285]:
##### Identifying trips with one agency

In [218]:
# cols_to_keep = ["transit_submode","unique_agencies", "n","n_modes_taken"]

In [217]:
# chart = (alt.Chart(df)
#              .mark_circle(size=100)
#              .encode(
#                  x=alt.X("n_modes_taken", title="Number of Modes taken"),
#                  y=alt.Y("n", title="Number of Trips"),
#                  color = alt.Color("agency_count", title="Number of Unique Agencies",
#                                   scale=alt.Scale(
#                                       range=cp.CALITP_DIVERGING_COLORS,
#                                       domain=df["agency_count"].unique().tolist())
#             ),
#             tooltip=cols_to_keep)
#         .properties(title = ("Transit Trips Agency Breakdown"), width=500,
#     height=300)
#         )
# chart                    

In [74]:
# (df>>filter(_.agency_count==1)>>arrange(-_.n))

In [287]:
chart = (alt.Chart((df>>filter(_.agency_count==1)))
             .mark_bar()
             .encode(
                 x=alt.X("unique_agencies", title="Agency"),
                 y=alt.Y("n", title="Number of Modes taken"),
                 color = alt.Color("n_modes_taken", title="Number of Trips",
                                  scale=alt.Scale(
                                      range=cp.CALITP_SEQUENTIAL_COLORS,)),
            tooltip=cols_to_keep)
        .properties(title = ("Transit Trips With Only One Agency"), width=800,
    height=500)
        )
chart           

alt.Chart(...)

In [288]:
display(HTML("<h4><strong>Most Common Agency Combination</strong></h4>")) 

(df>>arrange(-_.n)>>select(_.transit_agency)).head(5)

,transit_agency
0,AC TRANSIT
12,"San Francisco Municipal Transportation Agency, AC TRANSIT"
8,"AC TRANSIT, San Francisco Municipal Transportation Agency"
6,"AC TRANSIT, AC TRANSIT"
75,"AC TRANSIT, San Francisco Municipal Transportation Agency"


In [289]:
columns_to_keep = ["AC TRANSIT",
                   "San Francisco Bay Ferry",
                   "SolTrans",
                   "Caltrain",
                   "SamTrans",
                   "The S",
                   "Fairfield and Suisun Transit",
                   "Sacramento Regional Transit",
                   "Marin Transit",
                   "Tri Delta Transit",
                   "San Francisco Municipal Transportation Agency",
                   "Amtrak",
                   "San Joaquin Regional Transit District (RTD)",
                   "Bay Area Rapid Transit",
                   "Santa Cruz Metro",
                   "VTA"
                  ]

In [291]:
agency_indv_count.sample()

,transit_agency,AC TRANSIT,nan,Fairfield and Suisun Transit,SolTrans,The S,SamTrans,Caltrain,San Francisco Bay Ferry,Sacramento Regional Transit,Marin Transit,Tri Delta Transit,Amtrak,San Joaquin Regional Transit District (RTD),San Francisco Municipal Transportation Agency,Santa Cruz Metro,Bay Area Rapid Transit,VTA,unique_agencies
180902,AC TRANSIT,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AC TRANSIT


In [292]:
df_agencies = agency_indv_count.loc[:, columns_to_keep]

In [293]:
df_agencies = pd.DataFrame(df_agencies).transpose().reset_index().rename(columns={'index':'agency'})

In [294]:
df_agencies

,agency,6314,6776,7515,7706,9097,9956,9991,10992,11413,13919,14351,14866,15327,15942,15943,17018,17272,17719,17992,18072,18269,18270,19618,20564,21261,22111,22200,22248,22315,22316,22322,22324,22325,22328,22386,22400,22450,22469,22612,22744,22751,22803,22873,24110,24548,24741,24760,24832,25421,25795,27722,31244,31247,31507,32315,34084,35555,37800,37911,38125,39606,40907,40918,40974,40977,40998,42618,45516,45703,46200,46208,46414,46885,51311,51373,53198,53379,53468,53495,58120,60997,61552,63543,72144,73817,75190,75240,75276,75415,75698,75815,75839,80574,80588,80719,82067,82657,82678,84270,84276,85840,86191,87860,90356,90647,90650,91131,91155,94554,97640,97772,97951,98148,98150,98266,104234,104303,104304,104305,104306,104619,104914,109015,109058,109099,112101,112106,114150,114292,114366,114373,115048,115274,115277,115309,115310,115323,115329,115389,115486,115488,115679,115884,116031,116175,116294,116322,116701,116717,116722,116723,116758,116782,117119,117140,119455,119465,119475,125222,127079,127607,127900,127908,127921,127928,127947,128322,128341,128517,128712,129011,129015,130364,131421,132258,136558,137732,138045,138175,138177,138193,138212,138600,144190,144208,144210,144221,144281,144344,144536,150784,157527,158951,165314,169051,173574,173640,174057,174059,174073,174075,174076,174220,174231,174252,174282,174284,174318,174338,174351,174352,174371,174376,174394,174396,174410,174413,174424,174427,174431,174435,174436,174442,174445,174447,174453,174457,174461,174462,174468,174469,174479,174495,174501,174507,174512,174515,174516,174520,174522,174527,174532,174539,174544,174546,174557,174565,174583,174589,...,554774,554776,554777,554790,554791,554798,554803,554814,554820,554822,554838,554846,554905,554916,554982,554992,554995,555007,555011,555039,555044,555047,555062,555066,555075,555108,555113,555116,555119,555120,555121,555122,555123,555124,555125,555126,555128,555129,555132,555170,555195,555210,555211,555212,555222,555232,555235,555242,555260,555262,555286,555292,555309,555319,555338,555347,555364,555374,555391,555397,555433,555448,555455,555460,555608,555615,555659,555723,555796,556927,556929,556977,557004,557013,557020,557334,558239,558820,558950,559126,559130,559170,559197,559302,559327,559328,559343,559374,559382,559421,559532,559578,559623,559629,559630,559634,559636,559637,559638,559676,559695,559764,559774,559786,559790,559822,559823,559829,559846,559877,559880,559891,559892,559893,559894,559895,559897,559898,559900,559911,559929,559955,559978,559985,560030,560078,560123,560157,560168,560224,560244,560668,560682,560884,560895,560899,560900,560903,561359,561560,561593,561638,561640,561641,562114,562422,562435,562449,562466,562481,562504,562512,562544,562644,562649,562657,562664,562668,562674,562686,562697,562698,562702,562703,562714,562760,562763,562768,562799,562801,562802,562803,562805,562809,562812,562814,562815,562817,562818,562820,562824,562826,562827,562830,562833,562838,562845,562901,563209,563598,563610,563693,564530,565065,566230,566321,569501,570273,571259,571458,571464,571482,572527,572940,577860,579492,580859,581626,581663,582985,583213,589739,594654,594665,599627,603740,606355,610612,611097,614566,618762,621188,631490,632820,635259,636145,636158,636162,643970,647810,653707,653714,654910,655077,655097,655143,655177,655348,655423,655555,655878,656246,656542,662436,662905,663640,669529,669567,669894,670345
0,AC TRANSIT,0,1,1,1,1,1,1,2,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,2,2,1,0,2,1,1,0,1,0,0,0,1,1,1,0,0,1,1,2,2,1,1,1,0,1,1,1,2,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1,3,1,1,1,1,1,0,1,1,1,1,1,2,0,0,0,0,1,1,1,1,1,1,0,1,2,1,1,1,1,1,1,0,1,0,0,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,2,1,2,1,2,1,1,1,...,0,0,0,1,1,2,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,1,0,0,0,

In [295]:
df_agencies['n_trips'] = df_agencies[list(df_agencies.columns)].sum(axis=1)

/tmp/ipykernel_305/2517619377.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_agencies['n_trips'] = df_agencies[list(df_agencies.columns)].sum(axis=1)


In [296]:
df_agencies = df_agencies>>select(_.agency, _.n_trips)

In [302]:
tooltip_cols = ["agency", "n_trips"]

In [303]:
df_agencies

,agency,n_trips
0,AC TRANSIT,5438
1,San Francisco Bay Ferry,4
2,SolTrans,54
3,Caltrain,50
4,SamTrans,72
5,The S,6
6,Fairfield and Suisun Transit,1
7,Sacramento Regional Transit,9
8,Marin Transit,4
9,Tri Delta Transit,1


In [304]:
chart = (alt.Chart(df_agencies)
        .mark_bar()
        .encode(
            x=alt.X("agency", title = "Agency Name"),
            y=alt.Y("n_trips", title= "Number of boardings reported for trips (One person taking two AC Transit trips will count as 2)"),
            color=alt.Color("n_trips", scale=alt.Scale(range = cp.CALITP_SEQUENTIAL_COLORS)),
            tooltip = tooltip_cols)
      .properties(title = "Number of Times an Agency was used for Trip taking over Bay Bridge",
    width=800,
    height=500)
        )
chart 

alt.Chart(...)

In [305]:
display(HTML("<h4><strong>Trips by Resident Type</strong></h4>")) 


In [83]:
trip_by_res_type = (r_trips
 >>group_by(_.primary_mode,_.trip_taker_resident_type)
 >>summarize(number_trips = _.activity_id.nunique())
 >>arrange(_.primary_mode))

In [84]:
trip_by_res_type

,primary_mode,trip_taker_resident_type,number_trips
0,auto_passenger,core,367739
1,auto_passenger,visitor,23552
2,commercial,NaN,11897
3,on_demand_auto,core,24372
4,on_demand_auto,visitor,2581
5,private_auto,core,232486
6,private_auto,visitor,6204
7,public_transit,core,5986
8,public_transit,visitor,47


In [306]:
chart = (
        alt.Chart(trip_by_res_type)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode", title = "Mode"),
            y=alt.Y("number_trips", title = "Number of Trips"),
            color=alt.Color("trip_taker_resident_type", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
                                                                         domain=trip_by_res_type["trip_taker_resident_type"].unique().tolist())
            ),
            tooltip=trip_by_res_type.columns.tolist())
      .properties(
    width=800,
    height=500)
        )
chart 

alt.Chart(...)

In [86]:
display(HTML("<h4><strong>Unique Household Ids</strong></h4>")) 

In [87]:
r_trips>>count(_.trip_taker_household_id)>>arrange(-_.n)

,trip_taker_household_id,n
248686,,27897
262322,NaN,16384
110604,8194219563640751815,28
82299,6097771312917788596,27
74914,5552467881171814730,22
...,...,...
262312,9988489366890220188,1
262316,9991342309439330560,1
262317,9992490903429866665,1
262318,9992599661009943006,1


In [88]:
display(HTML("<h4><strong>Checking one household id</strong></h4>")) 


In [89]:
print(len(r_trips>>filter(_.trip_taker_household_id == 8194219563640751815)))
(r_trips>>filter(_.trip_taker_household_id == 8194219563640751815)).sample()

28


,origin_building_use,trip_duration_minutes,trip_taker_language,origin_trct_2020,trip_taker_household_id,trip_taker_sex,trip_taker_home_bgrp_2020,origin_st_2020,trip_taker_home_st_2020,trip_taker_race_ethnicity,transit_agency,transit_route,trip_taker_resident_type,vehicle_type,trip_taker_age,activity_id,destination_bgrp_2020,destination_building_use,trip_taker_wfh,destination_st_2020,destination_cty_2020,trip_taker_work_bgrp_2020,destination_trct_2020,trip_taker_household_income,origin_bgrp_2020,trip_taker_commute_mode,trip_taker_available_vehicles,primary_mode,previous_trip_purpose,trip_taker_building_type,transit_submode,trip_taker_work_st_2020,trip_taker_household_size,origin_cty_2020,destination_land_use,trip_taker_industry,trip_taker_tenure,vehicle_fuel_type,trip_taker_home_trct_2020,trip_taker_work_trct_2020,trip_taker_education,trip_start_time,trip_taker_individual_income,trip_taker_employment_status,trip_purpose,origin_land_use,trip_taker_work_cty_2020,trip_distance_miles,trip_taker_school_grade_attending,trip_taker_home_cty_2020,trip_taker_person_id,trip_end_time
656110,single_family,9,indo_european,"179.03 (San Francisco, CA)",8194219563640751815,male,"1 (Tract 179.03, San Francisco, CA)",California,California,asian_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,39.0,13541098643571256698,"1 (Tract 615.07, San Francisco, CA)",retail,unemployed_under_16_not_in_labor_force,California,San Francisco,Does not have work/school location,"615.07 (San Francisco, CA)",1200990.0,"1 (Tract 179.03, San Francisco, CA)",other_travel_mode,three_plus,private_auto,home,single_family,NaN,Does not have work/school location,10.0,San Francisco,mixed_use,naics72,renter,other_non_bev,"179.03 (San Francisco, CA)",Does not have work/school location,high_school,18:10:00,78984.0,unemployed,eat,single_family,Does not have work/school location,3.9,not_attending_school,San Francisco,12894946300265742193,18:19:12


In [90]:
## checking one household id
r_trips>>filter(_.trip_taker_household_id == 8194219563640751815)>>count(_.trip_taker_person_id, _.trip_taker_age,
                                                                         _.trip_taker_sex, _.trip_taker_household_size)

,trip_taker_person_id,trip_taker_age,trip_taker_sex,trip_taker_household_size,n
0,1325181746552868554,22.0,female,10.0,6
1,2310331066900902679,30.0,female,10.0,6
2,4746831412975349070,34.0,female,10.0,2
3,10066255028734967962,28.0,male,10.0,4
4,11058477761487230854,32.0,male,10.0,2
5,12894946300265742193,39.0,male,10.0,2
6,14409692340574959811,27.0,male,10.0,6


#### Adding in Replica's People Dataset

In [91]:
r_ppl = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_ppl}"))

/tmp/ipykernel_305/1951040339.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  r_ppl = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_ppl}"))


replica people dataframe sample

In [92]:
r_ppl.sample(5)

,person_id,work_st_2020,household_income,resident_type,language,available_vehicles,race_ethnicity,education,work_bgrp_2020,wfh,home_cty_2020,home_trct_2020,tenure,industry,work_trct_2020,school_grade_attending,building_type,commute_mode,employment_status,work_cty_2020,individual_income,home_st_2020,sex,household_size,home_bgrp_2020,household_id,age
49193,8682903469007956812,California,0.0,core,GQ_language,unknown_num_vehicles,asian_not_hispanic_or_latino,some_college,"1 (Tract 9820, Alameda, CA)",unemployed_under_16_not_in_labor_force,Alameda,"4228 (Alameda, CA)",GQ,naics72,"9820 (Alameda, CA)",undergraduate,GQ_structure,other_travel_mode,not_in_labor_force,Alameda,0.0,California,male,1.0,"1 (Tract 4228, Alameda, CA)",13526119813432325617,19.0
107374,17790302872816909985,California,138822.0,core,english,one,white_not_hispanic_or_latino,bachelors_degree,"1 (Tract 101.01, San Francisco, CA)",in_person,Alameda,"4041.01 (Alameda, CA)",renter,naics445299,"101.01 (San Francisco, CA)",not_attending_school,multiple_units,private_auto,employed,San Francisco,21861.0,California,female,2.0,"1 (Tract 4041.01, Alameda, CA)",9400661008842675581,31.0
102026,1840847354523713052,California,878322.0,core,spanish,three_plus,white_not_hispanic_or_latino,some_college,"1 (Tract 6133, San Mateo, CA)",remote,San Mateo,"6133 (San Mateo, CA)",owner,naics81,"6133 (San Mateo, CA)",undergraduate,single_family,worked_from_home,employed,San Mateo,10711.0,California,male,6.0,"1 (Tract 6133, San Mateo, CA)",1316252144924345956,22.0
16433,5785746260697777731,California,219234.0,core,spanish,three_plus,asian_not_hispanic_or_latino,bachelors_degree,"1 (Tract 615.01, San Francisco, CA)",in_person,Alameda,"4010 (Alameda, CA)",owner,naics6214,"615.01 (San Francisco, CA)",not_attending_school,several_units,auto_passenger,employed,San Francisco,52528.0,California,female,12.0,"2 (Tract 4010, Alameda, CA)",8580484783011523505,27.0
245251,16188142531198959675,California,142102.0,core,indo_european,three_plus,two_races_not_hispanic_or_latino,some_college,"2 (Tract 177, San Francisco, CA)",employed_not_working,Contra Costa,"3382.04 (Contra Costa, CA)",owner,naics451110,"177 (San Francisco, CA)",undergraduate,single_family,public_transit,employed,San Francisco,8744.0,California,female,4.0,"2 (Tract 3382.04, Contra Costa, CA)",16409011573581227320,20.0


In [93]:
r_ppl>>group_by(_.resident_type)>>summarize(_unique_ids = _.person_id.nunique())

,resident_type,_unique_ids
0,core,278158
1,visitor,23034


In [94]:
(r_ppl
 >>filter(_.home_bgrp_2020!="Visitor (no home location)")
 >>group_by(_.commute_mode, _.sex)
 >>summarize(
     n_ppl = _.person_id.nunique(),
     avg_h_income = _.household_income.mean(),
     avg_p_income = _.individual_income.mean(),
     avg_age = _.age.mean())
)

,commute_mode,sex,n_ppl,avg_h_income,avg_p_income,avg_age
0,auto_passenger,female,10908,175717.982765,70255.626054,42.234507
1,auto_passenger,male,11605,177241.394916,82072.291685,42.185523
2,biking,female,234,248565.141026,82592.418803,36.085470
3,biking,male,527,219563.083491,120152.859583,38.426945
4,other_travel_mode,female,36176,126129.721003,21413.682939,55.161599
5,other_travel_mode,male,26290,116775.549106,33563.078813,54.328262
6,private_auto,female,45312,186499.659406,76492.005341,42.626302
7,private_auto,male,59762,190643.201265,100712.386918,42.872093
8,public_transit,female,22008,193540.921347,84074.072110,41.921937
9,public_transit,male,26480,207838.001699,125445.942485,42.259290


In [95]:
(r_trips
 >>filter(_.trip_taker_home_bgrp_2020!="Visitor (no home location)")
 >>group_by(_.primary_mode, _.trip_taker_sex)
 >>summarize(
     n_ppl = _.trip_taker_person_id.nunique(),
     avg_h_income = _.trip_taker_household_income.mean(),
     avg_p_income = _.trip_taker_individual_income.mean(),
     avg_age = _.trip_taker_age.mean())
)
     

,primary_mode,trip_taker_sex,n_ppl,avg_h_income,avg_p_income,avg_age
0,auto_passenger,female,99350,173666.662228,65104.398490,45.064633
1,auto_passenger,male,105359,187590.985442,97296.509379,43.991868
2,commercial,NaN,1,NaN,NaN,NaN
3,on_demand_auto,female,9051,186226.833189,79990.395102,42.900399
4,on_demand_auto,male,10159,202467.780725,111915.392968,42.436605
5,private_auto,female,68268,174867.937251,62299.064752,45.316785
6,private_auto,male,75223,182301.952228,90322.373466,44.265320
7,public_transit,female,2256,186720.499643,82496.554922,39.104494
8,public_transit,male,2535,205334.441232,114041.914833,38.312382


### Read in Streetlight Data: 2022

In [96]:
streetlight = "streetlight_bay_bridge_corridor_study_corridor_study.csv"

In [97]:
sl_data = to_snakecase(pd.read_csv(f"{GCS_PATH}{streetlight}"))

In [98]:
sl_data.sample(5)

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
134,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1020456908,San Francisco – Oakland Bay Bridge / 23874736 / 1,Motorway,0.077,yes,221,WEST,no,1: Weekday (M-F),14: Mid-Day (10am-4pm),48401,52,5,68.747,3719.29,1.33,True,67
453,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1050419851,San Francisco – Oakland Bay Bridge / 179235221 / 1,Motorway,0.004,yes,38,EAST,no,0: All Days (M-Su),03: 1am (1am-2am),1726,66,2,67.840,7.44,1.03,False,75
1572,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1133975975,San Francisco – Oakland Bay Bridge / 929579737 / 1,Motorway,0.273,yes,215,WEST,no,1: Weekday (M-F),12: 9am (9am-10am),8910,38,30,68.634,2436.35,1.83,True,59
472,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1050419851,San Francisco – Oakland Bay Bridge / 179235221 / 1,Motorway,0.004,yes,38,EAST,no,0: All Days (M-Su),22: 5pm (5pm-6pm),10393,42,2,67.840,44.83,1.62,True,57
1534,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1133975975,San Francisco – Oakland Bay Bridge / 929579737 / 1,Motorway,0.273,yes,215,WEST,no,0: All Days (M-Su),04: 2am (2am-3am),855,68,14,68.634,233.79,1.01,False,75


#### Data Explorations: Streetlight

In [99]:
len(sl_data)

1710

In [100]:
sl_data>>count(_.zone_id, _.zone_name)

,zone_id,zone_name,n
0,1005898592,San Francisco – Oakland Bay Bridge / 8921938 / 1,90
1,1020456908,San Francisco – Oakland Bay Bridge / 23874736 / 1,90
2,1022959400,San Francisco – Oakland Bay Bridge / 661905446 / 1,90
3,1032819756,San Francisco – Oakland Bay Bridge / 11415208 / 1,90
4,1036056766,San Francisco – Oakland Bay Bridge / 52721870 / 1,90
5,1050419851,San Francisco – Oakland Bay Bridge / 179235221 / 1,90
6,1093181568,San Francisco – Oakland Bay Bridge / 50691047 / 1,90
7,1112052563,San Francisco – Oakland Bay Bridge / 236348360 / 1,90
8,1112062603,San Francisco – Oakland Bay Bridge / 236348361 / 1,90
9,1114495634,I 80 / 395419317 / 1,90


In [101]:
sl_data>>filter(_.zone_id==1133975975)>>count(_.day_part)

,day_part,n
0,00: All Day (12am-12am),3
1,01: 12am (12am-1am),3
2,02: Early AM (12am-6am),3
3,03: 1am (1am-2am),3
4,04: 2am (2am-3am),3
5,05: 3am (3am-4am),3
6,06: 4am (4am-5am),3
7,07: 5am (5am-6am),3
8,08: 6am (6am-7am),3
9,09: Peak AM (6am-10am),3


* The data here shows that each segment is broken out into hour time slots and then also aggregated to peak time periods. moving forward we will look at the `All Day` for `day_part` and `All Days` for `day_type`

In [102]:
sl_data_single_day = sl_data>>filter(_.day_part == '00: All Day (12am-12am)', _.day_type == '0: All Days (M-Su)')

In [103]:
len(sl_data_single_day)

19

In [104]:
sl_data_single_day>>count(_.zone_id, _.zone_name)

,zone_id,zone_name,n
0,1005898592,San Francisco – Oakland Bay Bridge / 8921938 / 1,1
1,1020456908,San Francisco – Oakland Bay Bridge / 23874736 / 1,1
2,1022959400,San Francisco – Oakland Bay Bridge / 661905446 / 1,1
3,1032819756,San Francisco – Oakland Bay Bridge / 11415208 / 1,1
4,1036056766,San Francisco – Oakland Bay Bridge / 52721870 / 1,1
5,1050419851,San Francisco – Oakland Bay Bridge / 179235221 / 1,1
6,1093181568,San Francisco – Oakland Bay Bridge / 50691047 / 1,1
7,1112052563,San Francisco – Oakland Bay Bridge / 236348360 / 1,1
8,1112062603,San Francisco – Oakland Bay Bridge / 236348361 / 1,1
9,1114495634,I 80 / 395419317 / 1,1


In [105]:
sl_data_single_day>>filter(_.zone_id==1133975975)

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
1530,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1133975975,San Francisco – Oakland Bay Bridge / 929579737 / 1,Motorway,0.273,yes,215,WEST,no,0: All Days (M-Su),00: All Day (12am-12am),128139,49,23,68.634,35038.3,1.39,True,67


In [106]:
sl_data_single_day.sample(3)

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
180,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1022959400,San Francisco – Oakland Bay Bridge / 661905446 / 1,Motorway,1.939,yes,40,EAST,no,0: All Days (M-Su),00: All Day (12am-12am),131167,49,149,64.627,254292.0,1.32,True,61
1440,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1132861884,I 80 / 236348365 / 1,Motorway,0.006,yes,85,EAST,no,0: All Days (M-Su),00: All Day (12am-12am),128581,63,2,70.532,725.2,1.12,False,72
1620,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1135393455,I 80 / 781148922 / 1,Motorway,0.256,yes,265,WEST,no,0: All Days (M-Su),00: All Day (12am-12am),104573,53,18,67.716,26778.5,1.28,True,67


In [107]:
sl_data_single_day>>group_by(_.zone_cardinal_direction, _.day_type, _.day_part)>>summarize(avg_speed= _.avg_segment_speed__mph_.mean(),
                                                                                          avg_travel_time = _.avg_segment_travel_time__sec_.mean(),
                                                                                          avg_volume = _.average_daily_segment_traffic__stl_volume_.mean(),
                                                                                          sum_volume = _.average_daily_segment_traffic__stl_volume_.sum())

,zone_cardinal_direction,day_type,day_part,avg_speed,avg_travel_time,avg_volume,sum_volume
0,EAST,0: All Days (M-Su),00: All Day (12am-12am),55.900000,32.600000,139682.300000,1396823
1,WEST,0: All Days (M-Su),00: All Day (12am-12am),50.333333,41.777778,123029.777778,1107268


In [108]:
sl_peaks = (sl_data>>filter(
                            _.day_part != "01: 12am (12am-1am)",
                            _.day_part != "03: 1am (1am-2am)",
                            _.day_part != "04: 2am (2am-3am)",
                            _.day_part != "05: 3am (3am-4am)",
                            _.day_part != "06: 4am (4am-5am)",
                            _.day_part != "07: 5am (5am-6am)",
                            _.day_part != "08: 6am (6am-7am)",
                            _.day_part != "10: 7am (7am-8am)",
                            _.day_part != "11: 8am (8am-9am)",
                            _.day_part != "12: 9am (9am-10am)",
                            _.day_part != "13: 10am (10am-11am)",
                            _.day_part != "15: 11am (11am-12noon)",
                            _.day_part != "16: 12pm (12noon-1pm)",
                            _.day_part != "17: 1pm (1pm-2pm)",
                            _.day_part != "18: 2pm (2pm-3pm)",
                            _.day_part != "19: 3pm (3pm-4pm)",
                            _.day_part != "20: 4pm (4pm-5pm)",
                            _.day_part != "22: 5pm (5pm-6pm)",
                            _.day_part != "23: 6pm (6pm-7pm)",
                            _.day_part != "24: 7pm (7pm-8pm)",
                            _.day_part != "25: 8pm (8pm-9pm)",
                            _.day_part != "27: 9pm (9pm-10pm)",
                            _.day_part != "28: 10pm (10pm-11pm)",
                            _.day_part !="29: 11pm (11pm-12am)"))

In [109]:
sl_hourly = (sl_data>>filter(_.day_part != "00: All Day (12am-12am)",
                            _.day_part != "02: Early AM (12am-6am)",
                            _.day_part != "09: Peak AM (6am-10am)",
                            _.day_part != "14: Mid-Day (10am-4pm)",
                            _.day_part != "21: Peak PM (4pm-8pm)", _.day_part !="26: Late PM (8pm-12am)"))

In [110]:
sl_peaks_agg = sl_peaks>>group_by(_.zone_cardinal_direction,
    _.day_type, _.day_part)>>summarize(avg_speed= _.avg_segment_speed__mph_.mean(),
                                                                                          avg_travel_time = _.avg_segment_travel_time__sec_.mean(),
                                                                                          avg_volume = _.average_daily_segment_traffic__stl_volume_.mean(),
                                                                                          sum_volume = _.average_daily_segment_traffic__stl_volume_.sum())

In [111]:
sl_peaks_agg.sample()

,zone_cardinal_direction,day_type,day_part,avg_speed,avg_travel_time,avg_volume,sum_volume
5,EAST,0: All Days (M-Su),26: Late PM (8pm-12am),59.6,29.5,23240.7,232407


In [112]:
chart = (
        alt.Chart(sl_peaks_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("avg_speed"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_peaks_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_peaks_agg.columns.tolist(),
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [113]:
chart = (
        alt.Chart(sl_peaks_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("avg_speed"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_peaks_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_peaks_agg.columns.tolist(),
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [114]:
sl_hourly_agg = sl_hourly>>filter(_.day_type=="0: All Days (M-Su)")>>group_by(_.zone_cardinal_direction,
    _.day_type, _.day_part)>>summarize(avg_speed= _.avg_segment_speed__mph_.mean(),
                                                                                          avg_travel_time = _.avg_segment_travel_time__sec_.mean(),
                                                                                          avg_volume = _.average_daily_segment_traffic__stl_volume_.mean(),
                                                                                          sum_volume = _.average_daily_segment_traffic__stl_volume_.sum())

In [115]:
sl_hourly_agg.sample()

,zone_cardinal_direction,day_type,day_part,avg_speed,avg_travel_time,avg_volume,sum_volume
6,EAST,0: All Days (M-Su),08: 6am (6am-7am),66.4,26.1,3067.0,30670


In [116]:
chart = (
        alt.Chart(sl_hourly_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("avg_speed"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_hourly_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_hourly_agg.columns.tolist(),
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [117]:
chart = (
        alt.Chart(sl_hourly_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("sum_volume"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_hourly_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_hourly_agg.columns.tolist(),
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)